In [199]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

In [200]:
dec_2016 = pd.read_csv('data/raw_data/Video_Games_Sales_as_at_22_Dec_2016.csv', header = 0, index_col = None)
jan_2017 = pd.read_csv('data/raw_data/Video_Game_Sales_as_of_Jan_2017.csv')
ign = pd.read_excel('data/raw_data/ign.csv.xlsx', header = 0, index_col = 0, sheetname = "ign.csv")
steam = pd.read_csv('data/raw_data/steam-200k.csv', header = None, index_col = None)

In [201]:
''' Returns the given dataset's titles in the form of a dictionary with release year as the keys. '''
def get_years_dict(df, year_feature_name, title_feature_name):
    years_dict = {}
    for year in df[year_feature_name]:
        if math.isnan(year):
            continue
        years_dict[year] = df[df[year_feature_name] == year][title_feature_name].tolist()
    return years_dict

In [202]:
''' Combines the two datasets and returns a dictionary of the union set, grouped by release year. '''
def union(df1, df2, yrs_label1, yrs_label2):
    yrs_dict1 = get_years_dict(df1, yrs_label1, 'Name')
    yrs_dict2 = get_years_dict(df2, yrs_label2, 'Name')
    
    # Add empty lists for years that either dicts have but the other does not.
    for year in yrs_dict1:
        if year not in yrs_dict2:
            yrs_dict2[year] = []
    for year in yrs_dict2:
        if year not in yrs_dict1:
            yrs_dict1[year] = []
    
    union = {}
    for year in yrs_dict1.keys():
        union[year] = yrs_dict1[year] + list(set(yrs_dict2[year]) - set(yrs_dict1[year]))
    
    return union

In [ ]:
''' Returns the intersection set of df1 and df2. '''
def intersection(df1, df2, title_feature_name1, title_feature_name2):
    df1 = df1.rename(index=str, columns={title_feature_name1 : "Name"})
    df2 = df2.rename(index=int, columns={title_feature_name2 : "Name"})
    intersection = df2.set_index("Name").join(df1.set_index("Name"), how='inner')
    return intersection

In [ ]:
intersection = intersection(ign, steam, 'title', 1)
union = union(dec_2016, jan_2017, 'Year_of_Release', 'Year_of_Release')

for year in union.keys():
    pd.DataFrame(union[year]).to_csv('data/time_series_data/' + str(year) + '.csv')

print 'Hello!'